In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
import os

In [3]:
print(os.listdir("../input"))

['test1', 'train']


In [4]:
print(os.listdir("../"))

['FYP face recog', 'FYP-P1', 'input', 'MLDM', 'NCP', 'NCP-Project', 'NLP', 'PDC', 'SICP', 'test', 'train']


In [5]:
if not os.path.exists("../train"):
    os.mkdir("../train")
if not os.path.exists("../train/A"):
    os.mkdir("../train/A")
if not os.path.exists("../train/B"):
    os.mkdir("../train/B")

In [6]:
import shutil
from tqdm import tqdm

In [7]:
for i in tqdm(os.listdir("../input/train")):
#     print(i)
    if i.split(".")[0] == "A":
        shutil.copy2(os.path.join("../input/train",i),os.path.join("../train/A",i))
    elif i.split(".")[0] == "B":
        shutil.copy2(os.path.join("../input/train",i),os.path.join("../train/B",i))

100%|█████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 833.46it/s]


In [8]:
if not os.path.exists("../test"):
    os.mkdir("../test")
if not os.path.exists("../test/A"):
    os.mkdir("../test/A")
if not os.path.exists("../test/B"):
    os.mkdir("../test/B")

In [9]:
import keras
from keras.models import Model
from keras.layers import Dense
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image

In [10]:
os.listdir("../train")

['A', 'B']

In [11]:
trdata = ImageDataGenerator()
traindata = trdata.flow_from_directory(directory="../train",target_size=(224,224))

Found 23 images belonging to 2 classes.


In [12]:
tsdata = ImageDataGenerator()
testdata = tsdata.flow_from_directory(directory="../test", target_size=(224,224))

Found 4 images belonging to 2 classes.


In [13]:
from keras.applications.vgg16 import VGG16


In [14]:
vggmodel = VGG16(weights='imagenet', include_top=True)

In [15]:
vggmodel.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [16]:
for layers in (vggmodel.layers):
    print(layers)
    layers.trainable = False

In [17]:
X= vggmodel.layers[-2].output

In [18]:
X

<KerasTensor: shape=(None, 4096) dtype=float32 (created by layer 'fc2')>

In [19]:
predictions = Dense(2, activation="softmax")(X)

In [20]:
model_final = Model(inputs = vggmodel.input, outputs = predictions)

In [21]:
model_final.compile(loss = "categorical_crossentropy", optimizer = tf.keras.optimizers.SGD(lr=0.0001, momentum=0.9), metrics=["accuracy"])

C:\Users\shakt\anaconda3\envs\vggfacemodel\lib\site-packages\keras\optimizer_v2\optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


In [22]:
model_final.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [23]:
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping

In [24]:
os.listdir("../")

['FYP face recog',
 'FYP-P1',
 'input',
 'MLDM',
 'NCP',
 'NCP-Project',
 'NLP',
 'PDC',
 'SICP',
 'test',
 'train']

In [25]:
checkpoint = ModelCheckpoint("vgg16_1.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=40, verbose=1, mode='auto')

In [26]:
hist = model_final.fit_generator(generator= traindata, steps_per_epoch= 2, epochs= 5, validation_data= testdata, validation_steps=1, callbacks=[checkpoint,early])

C:\Users\shakt\anaconda3\envs\vggfacemodel\lib\site-packages\keras\engine\training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/5
2/2 [==============================] - 276s 2s/step - loss: 1.8240 - accuracy: 0.3043 - val_loss: 1.5609 - val_accuracy: 0.2500


In [27]:
model_final.save_weights("vgg16_1.h5")


In [28]:
for e,i in enumerate(os.listdir("../input/test1")):
    print(i)
    output=[]
    img = image.load_img(os.path.join("../input/test1",i),target_size=(224,224))
    img = np.asarray(img)
    img = np.expand_dims(img, axis=0)
    output = model_final.predict(img)
    print(output)
#     if output[0][0] > output[0][1]:
#         print("A")
# #         df["id"][e]=i
# #         df["label"][e]="cat"
#     else:
#         print("B")
# #         df["id"][e]=i
# #         df["label"][e]="dog

Shakthi.jpg
[[0.34123662 0.6587634 ]]
shakthi2.jpg
[[0.909077   0.09092304]]
sk.jpg
[[0.03056837 0.9694316 ]]
skandan.jpg
[[0.931395   0.06860507]]
someguy.jpg
[[0.9123335  0.08766655]]
